In [ ]:
from datasets import load_dataset
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

import faiss
import numpy as np



In [4]:
dataset = load_dataset("qiaojin/PubMedQA",'pqa_artificial')['train']

In [5]:
mini_data = dataset.select(range(20000))

In [10]:
all_contexts = []
for data in tqdm(mini_data):
    contexts = data['context']['contexts']
    all_contexts.extend(contexts)

100%|██████████| 20000/20000 [00:02<00:00, 7381.51it/s]


In [11]:
len(all_contexts)

62380

In [13]:
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # Lightweight, fast
embeddings = encoder.encode(all_contexts, show_progress_bar=True, convert_to_tensor=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1950 [00:00<?, ?it/s]

In [15]:
type(embeddings)

torch.Tensor

In [22]:
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

In [23]:
index.add(embeddings.cpu().detach().numpy())


In [24]:
faiss.write_index(index, "pubmed_faiss.index")

In [26]:
import json
with open("pubmed_contexts.json", "w") as f:
    json.dump(all_contexts, f)

In [27]:
query = "Does aspirin help prevent heart attack?"
query_vec = encoder.encode([query])

In [29]:
D, I = index.search(np.array(query_vec), k=5)  # Get top-5 similar
with open("pubmed_contexts.json") as f:
    context_list = json.load(f)

results = [context_list[i] for i in I[0]]
print(results)

['Aspirin inhibits the cyclooxygenase-1 (COX-1) mediated thromboxane A2 synthesis. Despite COX-1 inhibition, in patients with coronary artery disease (CAD), platelets can be activated through other mechanisms, like activation by thrombin.', 'To evaluate the role of reticulated platelets in the antiplatelet effects of aspirin.', 'Antiplatelet therapy has been proven to be effective for both primary and secondary prevention of myocardial infarction, stroke, and cardiovascular death. However, a significant proportion of patients treated with aspirin experience ischemic events. A number of prospective studies have demonstrated that decreased responsiveness to antiplatelet therapy as measured by various methods, is strongly associated with an increase in clinical events. Our objective was to characterize platelet function in patients presenting with chest pain using a point-of-care assay, PFA-100 and correlating results to traditional platelet aggregometry to determine if patients with aspi